In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time

# Query for Google search
search_query = "site:instagram.com/p/ #dentist #bengaluru"

# Configure Chrome options for incognito mode and user-agent
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")

# Initialize WebDriver
driver = webdriver.Chrome(options=chrome_options)

try:
    # Open Google and perform the search
    driver.get("https://www.google.com")
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "q")))
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(search_query)
    search_box.send_keys(u'\ue007')  # Press Enter

    # Wait for results to load
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.yuRUbf a")))

    # Navigate through pages until the last page
    links = set()  # Use a set to avoid duplicate links
    while True:
        try:
            # Extract search result links from the current page
            results = driver.find_elements(By.CSS_SELECTOR, "div.yuRUbf a")
            page_links = {result.get_attribute("href") for result in results}
            links.update(page_links)

            print(f"Collected {len(page_links)} links from this page.")

            # Locate and click the "Next" button
            next_button = driver.find_element(By.ID, "pnnext")
            next_button.click()

            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.yuRUbf a")))

        except NoSuchElementException:
            print("No more pages to navigate. Reached the last page.")
            break
        except TimeoutException:
            print("Timeout while waiting for the next page to load.")
            break

    # Output collected links
    print(f"Total links collected: {len(links)}")
    with open("collected_links.txt", "w") as f:
        for link in links:
            f.write(link + "\n")

finally:
    # Close the browser
    driver.quit()


Collected 9 links from this page.
Collected 10 links from this page.
Collected 9 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 8 links from this page.
Collected 8 links from this page.
Collected 9 links from this page.
Collected 10 links from this page.
Collected 9 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 9 links from this page.
Collected 10 links from this page.
Collected 9 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 10 links from this page.
Collected 3 links from this page.
No more pages to navigate. Reached the last page.
Total links collected: 209


In [2]:
import re
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Load Instagram links from the file
with open("collected_links.txt", "r") as file:
    instagram_links = [line.strip() for line in file.readlines()]

# Instagram session ID
session_id = "61750047164%3AgkFp8EdO1q1I3F%3A25%3AAYfrQQk5JPLxXVdzZqt5Gu_7N9AnswNmJSK1FZ4ijw"

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(options=chrome_options)

# Log in using session ID
driver.get("https://www.instagram.com/")
driver.add_cookie({"name": "sessionid", "value": session_id, "domain": ".instagram.com"})
driver.refresh()

# Wait for the page to load after login
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    print("Logged in successfully!")
except TimeoutException:
    print("Failed to log in. Please check your session ID and try again.")
    driver.quit()
    exit()

# Prepare output file
output_file = "instagram_profile_urls.csv"
header = ["Original Link", "Profile URL"]

with open(output_file, "w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(header)

    # Scrape each Instagram link
    for link in instagram_links:
        try:
            driver.get(link)

            # Wait for the page to load
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))

            # Default to the original link
            profile_url = link
            if "/p/" in link or "/reel/" in link:
                try:
                    # Extract the first valid link from the post/reel page
                    username_element = driver.find_element(By.CSS_SELECTOR, "a[href*='/']").get_attribute("href")
                    profile_url_match = re.search(r"https://www.instagram.com/[^/]+/", username_element)
                    
                    if profile_url_match:
                        profile_url = profile_url_match.group(0)
                    else:
                        print(f"Profile URL pattern not found on page for {link}")
                        continue  # Skip this link and move to the next
                    
                except NoSuchElementException:
                    print(f"Could not extract profile URL from {link}")
                    continue

            # Validate and write the profile URL to CSV
            if "https://www.instagram.com/" in profile_url:
                writer.writerow([link, profile_url])
                print(f"Extracted profile URL: {profile_url}")
            else:
                print(f"Invalid profile URL for {link}: {profile_url}")

            # Avoid overwhelming Instagram
            time.sleep(2)

        except TimeoutException:
            print(f"Timeout while loading {link}")
        except NoSuchElementException:
            print(f"Could not find some elements on {link}")
        except Exception as e:
            print(f"Error while processing {link}: {e}")

# Close the browser
driver.quit()
print(f"Profile URL extraction completed. Data saved to {output_file}.")


Logged in successfully!
Extracted profile URL: https://www.instagram.com/bangalore_life_style/
Extracted profile URL: https://www.instagram.com/_ambikaneela_/
Extracted profile URL: https://www.instagram.com/drtrivikramcosmeticdentist/
Extracted profile URL: https://www.instagram.com/whatthefooooooood/
Extracted profile URL: https://www.instagram.com/thetatvaindia/
Profile URL pattern not found on page for https://www.instagram.com/p/C6iH9i_rPFd/
Extracted profile URL: https://www.instagram.com/bcledition1/
Extracted profile URL: https://www.instagram.com/thetatvaindia/
Extracted profile URL: https://www.instagram.com/drshetty83/
Extracted profile URL: https://www.instagram.com/indianewshd/
Extracted profile URL: https://www.instagram.com/gitamdeemeduniversity/
Extracted profile URL: https://www.instagram.com/medicaldialogues/
Extracted profile URL: https://www.instagram.com/manvithadentalclinic_mdc/
Profile URL pattern not found on page for https://www.instagram.com/p/DB0SIx_IoRx/?loc

Extracted profile URL: https://www.instagram.com/gitamdeemeduniversity/
Extracted profile URL: https://www.instagram.com/drrashmitha_rathnakar/
Extracted profile URL: https://www.instagram.com/gitamdeemeduniversity/
Extracted profile URL: https://www.instagram.com/thetatvaindia/
Extracted profile URL: https://www.instagram.com/geethaganta_/
Extracted profile URL: https://www.instagram.com/dr_srisupraja/
Profile URL pattern not found on page for https://www.instagram.com/p/C5Pv6KlNo_7/
Extracted profile URL: https://www.instagram.com/zubairalijung/
Extracted profile URL: https://www.instagram.com/dentamitraofficial/
Extracted profile URL: https://www.instagram.com/braces_quintessence_dental/
Extracted profile URL: https://www.instagram.com/gitamdeemeduniversity/
Extracted profile URL: https://www.instagram.com/f3news/
Extracted profile URL: https://www.instagram.com/dolly.mickey777/
Extracted profile URL: https://www.instagram.com/bangaloreepicure/
Extracted profile URL: https://www.ins

Profile URL extraction completed. Data saved to instagram_profile_urls.csv.
